In [30]:
import requests
from pprint import pprint

API_KEY = "Q4U1hSAXBTKf5LizcqlgjVaKFjjqqeZtAgScMour"
query = "banana, raw"

url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={query}&api_key={API_KEY}"
response = requests.get(url)
data = response.json()

#pprint(data)

# Get the first food item
food = data["foods"][0]
#pprint(food)
nutrients = {n["nutrientName"]: n["value"] for n in food["foodNutrients"]}

print(f"Food: {food['description']}")
print(f"Ingredient: {food.get('ingredients','N/A')}")
print(f"Energy (kcal): {nutrients.get('Energy', 'N/A')}")
print(f"Protein (g): {nutrients.get('Protein', 'N/A')}")
print(f"Fat (g): {nutrients.get('Total lipid (fat)', 'N/A')}")
print(f"Carbs (g): {nutrients.get('Carbohydrate, by difference', 'N/A')}")
print(f"Fiber (g): {nutrients.get('Fiber, total dietary', 'N/A')}")
print(f"Glucose (g): {nutrients.get('Glucose (dextrose)', 'N/A')}")


Food: Banana, raw
Ingredient: N/A
Energy (kcal): 97
Protein (g): 0.74
Fat (g): 0.28
Carbs (g): 22.71
Fiber (g): 1.7
Glucose (g): N/A


In [31]:
import requests

API_KEY = "Q4U1hSAXBTKf5LizcqlgjVaKFjjqqeZtAgScMour"
SEARCH_URL = "https://api.nal.usda.gov/fdc/v1/foods/search?api_key=" + API_KEY

def find_raw_food(query, data_types=None, page_size=25):
    """
    Search FDC and preferentially return Foundation / SR (raw) food.
    - query: food name, e.g. "banana"
    - data_types: list like ["Foundation","SR Legacy","Survey (FNDDS)"] or None to use defaults
    """
    if data_types is None:
        data_types = ["Foundation", "SR Legacy", "Survey (FNDDS)"]

    payload = {
        "query": query,
        "dataType": data_types,
        "pageSize": page_size,
        "pageNumber": 1
    }

    res = requests.post(SEARCH_URL, json=payload, timeout=10)
    res.raise_for_status()
    data = res.json()

    foods = data.get("foods", [])
    def looks_like_raw(f):
        # 1) prefer records that are not 'Branded' and have no ingredients text
        dtype = f.get("dataType", "").lower()
        if "branded" in dtype:
            return False
        # If 'ingredients' exists it's likely a manufactured product
        if f.get("ingredients"):
            return False
        # Many Foundation/SR raw items include 'raw' in description
        desc = (f.get("description") or "").lower()
        if "raw" in desc or "fresh" in desc or "uncooked" in desc:
            return True
        # fallback: prefer shorter descriptions (e.g., "Banana" vs long product names)
        if len(desc.split()) <= 3:
            return True
        return False

    # Prefer items that pass looks_like_raw()
    for f in foods:
        if looks_like_raw(f):
            return f

    # If none matched, return first non-branded; else return first result
    for f in foods:
        if "branded" not in (f.get("dataType","").lower()):
            return f

    return foods[0] if foods else None

# Example usage:
food = find_raw_food("tomato, raw")
if not food:
    print("No results")
else:
    print("Chosen item:", food.get("description"))
    # Extract common nutrients if present:
    nutrients = {n["nutrientName"]: n["value"] for n in food.get("foodNutrients", [])}
    print("Energy (kcal):", nutrients.get("Energy", "N/A"))
    print("Protein (g):", nutrients.get("Protein", "N/A"))
    print("Carbs (g):", nutrients.get("Carbohydrate, by difference", "N/A"))
    print("Fiber (g):", nutrients.get("Fiber, total dietary", "N/A"))


Chosen item: Tomatoes, raw
Energy (kcal): 20
Protein (g): 0.82
Carbs (g): 4.04
Fiber (g): 1.2


In [37]:
food_list = [
    # 🍎 Fruits (raw)
    "Apple, raw", "Banana, raw", "Orange, raw", "Lemon, raw", "Mango, raw",
    "Pineapple, raw", "Papaya, raw", "Watermelon, raw", "Melon, raw",
    "Strawberry, raw", "Blueberry, raw", "Raspberry, raw", "Grape, raw",
    "Cherry, raw", "Kiwi, raw", "Pear, raw", "Peach, raw", "Apricot, raw",
    "Plum, raw", "Fig, raw", "Pomegranate, raw", "Avocado, raw", "Dates, raw",
    "Coconut meat, raw",

    # 🥦 Vegetables (raw)
    "Tomato, raw", "Carrot, raw", "Potato, raw", "Sweet potato, raw",
    "Cucumber, raw", "Lettuce, raw", "Spinach, raw", "Kale, raw",
    "Broccoli, raw", "Cauliflower, raw", "Bell pepper, raw", "Onion, raw",
    "Garlic, raw", "Celery, raw", "Zucchini, raw", "Beetroot, raw",
    "Cabbage, raw", "Radish, raw", "Mushroom, raw", "Eggplant, raw",

    # 🌾 Grains & Cereals (uncooked)
    "Rice, white, uncooked", "Rice, brown, uncooked", "Oats, uncooked",
    "Quinoa, uncooked", "Wheat, whole, uncooked", "Barley, uncooked",
    "Corn, raw", "Buckwheat, uncooked",

    # 🫘 Legumes (raw / dried)
    "Lentils, raw", "Chickpeas, raw", "Kidney beans, raw", "Black beans, raw",
    "Peas, green, raw", "Soybeans, raw", "Mung beans, raw", "Pinto beans, raw",

    # 🥜 Nuts & Seeds (raw)
    "Almonds, raw", "Walnuts, raw", "Cashews, raw", "Hazelnuts, raw",
    "Pistachios, raw", "Sunflower seeds, raw", "Pumpkin seeds, raw",
    "Flax seeds, raw", "Chia seeds, raw", "Sesame seeds, raw",

    # 🥩 Animal Products (raw)
    "Chicken breast, raw", "Beef, raw", "Lamb, raw", "Pork, raw",
    "Egg, whole, raw", "Milk, raw", "Yogurt, plain, raw",

    # 🐟 Fish & Seafood (raw)
    "Salmon, raw", "Tuna, raw", "Sardine, raw", "Shrimp, raw",
    "Cod, raw", "Mackerel, raw"
]

dataset = []
for food in food_list:
    f = find_raw_food(food)
    if not f: continue
    nutrients = {}
    for n in f.get("foodNutrients", []):
        name = n.get("nutrientName")
        value = n.get("value")
        if name and value is not None:
            nutrients[name] = value
            
    dataset.append({
        "Food": f["description"],
        "Category": "auto",
        "kcal": nutrients.get("Energy","N/A"),
        "Protein": nutrients.get("Protein","N/A"),
        "Fat": nutrients.get("Total lipid (fat)","N/A"),
        "Carbs": nutrients.get("Carbohydrate, by difference","N/A"),
        "Fiber": nutrients.get("Fiber, total dietary","N/A"),
        "Glucose": nutrients.get("Glucose (dextrose)","N/A"),
        "FDC_ID": f.get("fdcId","N/A")
    })


In [38]:
dataset

[{'Food': 'Apple, raw',
  'Category': 'auto',
  'kcal': 61,
  'Protein': 0.17,
  'Fat': 0.15,
  'Carbs': 14.8,
  'Fiber': 2.1,
  'Glucose': 'N/A',
  'FDC_ID': 2709215},
 {'Food': 'Banana, raw',
  'Category': 'auto',
  'kcal': 97,
  'Protein': 0.74,
  'Fat': 0.28,
  'Carbs': 22.71,
  'Fiber': 1.7,
  'Glucose': 'N/A',
  'FDC_ID': 2709224},
 {'Food': 'Orange, raw',
  'Category': 'auto',
  'kcal': 50,
  'Protein': 0.92,
  'Fat': 0.14,
  'Carbs': 11.78,
  'Fiber': 2.2,
  'Glucose': 'N/A',
  'FDC_ID': 2709171},
 {'Food': 'Lemon, raw',
  'Category': 'auto',
  'kcal': 29,
  'Protein': 1.1,
  'Fat': 0.3,
  'Carbs': 9.32,
  'Fiber': 2.8,
  'Glucose': 'N/A',
  'FDC_ID': 2709168},
 {'Food': 'Mango, raw',
  'Category': 'auto',
  'kcal': 60,
  'Protein': 0.82,
  'Fat': 0.38,
  'Carbs': 14.98,
  'Fiber': 1.6,
  'Glucose': 'N/A',
  'FDC_ID': 2709242},
 {'Food': 'Pineapple, raw',
  'Category': 'auto',
  'kcal': 60,
  'Protein': 0.46,
  'Fat': 0.21,
  'Carbs': 14.09,
  'Fiber': 0.9,
  'Glucose': 'N/A',


In [39]:
import pandas as pd
df = pd.DataFrame(dataset)
df

,Food,Category,kcal,Protein,Fat,Carbs,Fiber,Glucose,FDC_ID
0,"Apple, raw",auto,61,0.17,0.15,14.80,2.1,N/A,2709215
1,"Banana, raw",auto,97,0.74,0.28,22.71,1.7,N/A,2709224
2,"Orange, raw",auto,50,0.92,0.14,11.78,2.2,N/A,2709171
3,"Lemon, raw",auto,29,1.10,0.30,9.32,2.8,N/A,2709168
4,"Mango, raw",auto,60,0.82,0.38,14.98,1.6,N/A,2709242
...,...,...,...,...,...,...,...,...,...
78,"Fish, tuna, raw",auto,109,24.40,0.49,0.00,0,N/A,2706308
79,Sardine sandwich,auto,234,17.87,7.96,21.87,1,N/A,2707033
80,"Crustaceans, shrimp, raw",auto,356,20.10,0.51,0.00,N/A,N/A,175179
81,"Fish, cod, Atlantic, raw",auto,343,17.80,0.67,0.00,0.0,N/A,171955


In [40]:
df.to_csv("raw_food_nutrition_dataset.csv", index=False)